In [ ]:
import pandas as pd

data = r"3993249.csv"

df = pd.read_csv(data)
df = df[df['STATION'] == 'USW00014935']
df.set_index('DATE', inplace=True)

df = df.loc['1964-04-01':]

In [ ]:
null_cols = df.isnull().sum()

#print(null_cols)

empty = null_cols[null_cols > 15000]

bad_columns = [c for c in list(empty.index) if 'WT' not in c]

#print(bad_columns)

condensed = df.drop(bad_columns, axis=1)

#print(condensed)

In [ ]:
condensed['TAVG'] = (condensed['TMAX'] + condensed['TMIN']) / 2

In [ ]:
weather_type_columns = [c for c in condensed.columns if 'WT' in c]
condensed[weather_type_columns] = condensed[weather_type_columns].fillna(0)
condensed

In [ ]:
condensed.to_csv('test.csv')

In [54]:
import numpy as np

new_df = condensed.loc['1984-01-01':'2024-12-31']

columns_to_drop = ['ACMH', 'ACSH', 'FMTM', 'PGTM', 'WDF5', 'WSF5', 'SNOW', 'SNWD']
columns_to_keep = [c for c in new_df.columns if c not in columns_to_drop]

new_df = new_df[columns_to_keep]
new_df[['WDF1', 'WDF2', 'WSF1', 'WSF2']] = new_df[['WDF1', 'WDF2', 'WSF1', 'WSF2']].fillna(0)

# combine the two remaining wind measurements into one 
new_df['WDF12'] = np.pi * (new_df['WDF1'] + new_df['WDF2']) / 180.0
new_df['WSF12'] = new_df['WSF1'] + new_df['WSF2']

# also include the U and V components of the wind
new_df['UWDF12'] = new_df['WSF12'] * np.cos(new_df['WDF12'])
new_df['VWDF12'] = new_df['WSF12'] * np.sin(new_df['WDF12'])

# use hundredths of an inch for precipitation measurement
new_df['PRCP'] = new_df['PRCP'] * 100.0

new_df.insert(0, 'DAY', pd.to_datetime(new_df.index).day_of_year)

new_df['DAY365'] = pd.to_datetime(new_df.index).day_of_year / 365.25
new_df['SIN'] = np.sin(2 * np.pi * new_df['DAY365'])
new_df['COS'] = np.cos(2 * np.pi * new_df['DAY365'])

new_df = new_df.drop(['WDF1', 'WDF2', 'WSF1', 'WSF2'], axis=1)
new_df = new_df.ffill()

new_df = new_df.drop(['STATION', 'NAME'], axis=1)
new_df.to_csv('grand_island.csv')
new_df.drop(weather_type_columns, axis=1).to_csv('grand_island_nwt.csv')

In [55]:
up_to_2000 = new_df.loc[:'2000-01-01']
up_to_2000

,DAY,AWND,PRCP,TMAX,TMIN,WT01,WT02,WT03,WT04,WT05,...,WT21,WT22,TAVG,WDF12,WSF12,UWDF12,VWDF12,DAY365,SIN,COS
DATE,,,,,,,,,,,,,,,,,,,,,
1984-01-01,1,8.50,3.0,40.0,26.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,33.0,4.886922,12.1,2.101143,-11.916174,0.002738,0.017202,0.999852
1984-01-02,2,11.41,0.0,36.0,19.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,27.5,3.839724,17.9,-13.712196,-11.505898,0.005476,0.034398,0.999408
1984-01-03,3,12.30,0.0,39.0,26.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,32.5,4.014257,21.0,-13.498540,-16.086933,0.008214,0.051584,0.998669
1984-01-04,4,11.63,0.0,40.0,33.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,36.5,5.410521,16.1,10.348881,-12.333316,0.010951,0.068755,0.997634
1984-01-05,5,11.86,0.0,45.0,36.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,40.5,5.934119,15.0,14.095389,-5.130302,0.013689,0.085906,0.996303
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1999-12-28,362,16.11,0.0,62.0,30.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,46.0,5.410521,30.0,19.283628,-22.981333,0.991102,-0.055879,0.998438
1999-12-29,363,9.17,0.0,68.0,25.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,46.5,4.537856,16.1,-2.795736,-15.855405,0.993840,-0.038696,0.999251
1999-12-30,364,6.93,0.0,54.0,23.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,38.5,5.759587,12.1,10.478907,-6.050000,0.996578,-0.021501,0.999769


In [56]:

ddd = up_to_2000.drop(weather_type_columns, axis=1).groupby(['DAY']).agg(['mean', 'std'])#.to_csv('grand_island_normals.csv')
ddd.columns = ['_'.join(col).strip() for col in ddd.columns.values]

#ddd.to_csv('grand_island_normals.csv')
total_means = new_df.drop(weather_type_columns, axis=1).agg(['mean', 'std'])

c = {}
for column in total_means.columns:
    if column == 'DAY':
        continue

    mean = total_means[column].loc['mean']
    std = total_means[column].loc['std']
    c[f'{column}_mean'] = mean
    c[f'{column}_std'] = std
    
print(c)

total_means = pd.DataFrame([c])
total_means.index = [9999]

pd.concat((ddd, total_means)).to_csv('grand_island_normals.csv')

{'AWND_mean': np.float64(11.091864983974359), 'AWND_std': np.float64(4.230069389399153), 'PRCP_mean': np.float64(7.125066773504273), 'PRCP_std': np.float64(25.468908342312798), 'TMAX_mean': np.float64(63.17821848290598), 'TMAX_std': np.float64(21.969177581267587), 'TMIN_mean': np.float64(39.72282318376068), 'TMIN_std': np.float64(19.866621093142225), 'TAVG_mean': np.float64(51.450520833333336), 'TAVG_std': np.float64(20.479551813128406), 'WDF12_mean': np.float64(3.7047107367035355), 'WDF12_std': np.float64(1.8372957519868833), 'WSF12_mean': np.float64(21.64308226495727), 'WSF12_std': np.float64(7.420523105557327), 'UWDF12_mean': np.float64(1.2569070706428394), 'UWDF12_std': np.float64(19.15899108477784), 'VWDF12_mean': np.float64(-2.574364987784377), 'VWDF12_std': np.float64(12.175540871069485), 'DAY365_mean': np.float64(0.5013947024342301), 'DAY365_std': np.float64(0.28869942461393744), 'SIN_mean': np.float64(5.061026781259483e-06), 'SIN_std': np.float64(0.7071127610397855), 'COS_mean